In [24]:
def package_url_sale(page):
    return 'https://www.zillow.com/homes/for_sale/new-york-city_rb/' + page

In [25]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import random
import json
import requests

price =[]
where =[]
bed = []
bath = []
size =[]
monthly = []
steet = []

In [26]:
for x in range(34,396): #loop through all pages
    url=package_url_rent(str(x))
    openurl= urllib.request.install_opener
    import urllib.request
    html = urllib.request.urlopen('https://www.zillow.com/homes/for_sale/new-york-city_rb/').read()   
    soup = BeautifulSoup(html,'html.parser')
    lst = soup.find_all(lambda tag: tag.has_attr('data-id'))
    for i in range(len(lst)):
        #price
        if lst[i].find_all('div',{'class':'list-card-price'})==[]:
            price.append('')
        else:
            price.append(lst[i].find_all('div',{'class':'list-card-price'})[0].string)
        #bedroom
        if(lst[i].find_all('ul',{'class':'list-card-details'})==[]):
            bed.append('')
        else:
            bed.append(lst[i].find_all('ul',{'class':'list-card-details'})[0].string)

print 

<function print>

In [27]:
import pandas as pd
import numpy as np
data = {'price':price,'where':where,'bed':bed, 'bath':bath, 'size':size,'furnished':0}
df = pd.DataFrame(index=data, columns=['price', 'where', 'bed', 'bath','size','furnished'])

def save_to_file(response):
    # saving response to `response.html`

    with open("response.html", 'w') as fp:
        fp.write(response.text)


def write_data_to_csv(data):
    # saving scraped data to csv.

    with open("properties-%s.csv" % (zipcode), 'wb') as csvfile:
        fieldnames = ['title', 'address', 'city', 'state', 'postal_code', 'price', 'facts and features', 'real estate provider', 'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)
            
def get_response(url):
    # Getting response from zillow.com.

    for i in range(5):
        response = requests.get(url, headers=get_headers())
        print("status code received:", response.status_code)
        if response.status_code != 200:
            # saving response to file for debugging purpose.
            save_to_file(response)
            continue
        else:
            save_to_file(response)
            return response
    return None

def get_data_from_json(raw_json_data):
    # getting data from json (type 2 of their A/B testing page)

    cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
    properties_list = []

    try:
        json_data = json.loads(cleaned_data)
        search_results = json_data.get('searchResults').get('listResults', [])
        for properties in search_results:
            address = properties.get('addressWithZip')
            property_info = properties.get('hdpData', {}).get('homeInfo')
            city = property_info.get('city')
            state = property_info.get('state')
            postal_code = property_info.get('zipcode')
            price = properties.get('price')
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            info = f'{bedrooms} bds, {bathrooms} ba ,{area} sqft'
            broker = properties.get('brokerName')
            property_url = properties.get('detailUrl')
            title = properties.get('statusText')

            data = {'address': address,
                    'city': city,
                    'state': state,
                    'postal_code': postal_code,
                    'price': price,
                    'facts and features': info,
                    'real estate provider': broker,
                    'url': property_url,
                    'title': title}
            properties_list.append(data)

        return properties_list

    except ValueError:
        print("Invalid json")
        return None
    
    def parse(zipcode, filter=None):
        url = create_url(zipcode, filter)
    response = get_response(url)

    if not response:
        print("Failed to fetch the page, please check `response.html` to see the response received from zillow.com.")
        return None

    parser = html.fromstring(response.text)
    search_results = parser.xpath("//div[@id='search-results']//article")
    argparser.add_argument('sort', nargs='?', help=sortorder_help, default='Homes For You')
    args = argparser.parse_args()
    zipcode = args.zipcode
    sort = args.sort
df.to_csv('rent-zillow-3.csv',encoding='utf-8')

In [28]:
import pandas as pd
import numpy as np
data = {'price':price,'where':where,'bed':bed, 'bath':bath, 'size':size,'furnished':0}
df = pd.DataFrame(index=data, columns=['price', 'where', 'bed', 'bath','size','furnished'])

#move from size to bath
cond=[]
for i in data['size']:
    if(i==''):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
df.loc[cond,'bath']= df.loc[cond,'size'] 
df.loc[cond,'size']=''

#move from bed to bath
cond=[]
for i in data['bed']:
    if(i=='' or i=='Furnished' or i=='studio'):
        cond.append(False)
    else:
        cond.append(i.split(" ")[1] in ('bath','baths'))
df.loc[cond,'bath']=df.loc[cond,'bed'] 
df.loc[cond,'bed']=''

#move from bath to bed
cond=[]
for i in data['bath']:
    if(i==''):
        cond.append(False)
    else:
        if(len(i.split(" "))==1):
            cond.append(True)
        else:
            if(i.split(" ")[1] in ('bath','baths')):
                cond.append(False)
            else:
                cond.append(True)
df.loc[cond,'bed']=df.loc[cond,'bath'] 
df.loc[cond,'bath']=''

#move from bed to size
cond=[]
for i in data['bed']:
    if(i=='' or i=='studio'):
        cond.append(False)
    else:
        if(i.split(" ")[1] in ('bed','beds')):
            cond.append(False)
        else:
            cond.append(True)
df.loc[cond,'size']=df.loc[cond,'bed'] 
df.loc[cond,'bed']=''


#replace blank with nan
data=df.applymap(lambda x: np.nan if x=='' else x)

#data
df.to_csv('rent-zillow.csv',encoding='utf-8')

In [29]:
#size to numeric
cond=data['size'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'size']=int(data['size'][i].split(" ")[0].replace(',',''))
#bath to numeric
cond=data['bath'].isnull()
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'bath']=float(data['bath'][i].split(" ")[0].replace('+',''))
#bed to numeric
cond=data['bed'].isnull()
data['bed']=data['bed'].replace('studio','0 bed')
for i in range(0,len(cond)):
    if (not cond[i]):
        df.loc[i,'bed']=float(data['bed'][i].split(" ")[0].replace(',','').replace('+',''))

df.to_csv('rent_zillow_2.csv')